In [1]:
import lightgbm as lgb
from sklearn import *
import pandas as pd
import numpy as np
pd.set_option('max_columns', None)

path = '../../Data/'
#from top scoring kernels and blends - for testing only
# sub1 = pd.read_csv('../input/best-ensemble-score-made-available-0-68/SHAZ13_ENS_LEAKS.csv')
# sub2 = pd.read_csv('../input/best-ensemble-score-made-available-0-67/SHAZ13_ENS_LEAKS.csv')

#standard
train = pd.read_csv(path + '/train.csv', index_col = 'ID')
# test = pd.read_csv(path + '/test.csv')
# print(train.shape, test.shape)

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected

In [7]:
from leak_rows import *
from leak_cols import *
from lag_list import *


def row_equal(df, r1, r2, lag):
#     if lag >= df.shape[1]:
#         return False
    return np.all(df.loc[r1].iloc[:-(lag)].values == df.loc[r2].iloc[lag:].values)
        
def check_leak_cols(df, leak_cols, leak_rows):
    # If df.shape[1] >= 3, we must have equal number between tow rows
    # Else if df.shape[1] == 2, we must have at least one row pair has equal number
#     dim = len(leak_cols)
#     has_equal_row_pair = False
    for i in range(len(leak_rows) - 1):
        r = leak_rows[i]
        r_next = leak_rows[i + 1]
        if not row_equal(df[leak_cols], r, r_next, LAG_LIST[i]):
            return False
#             print ('Leak cols are wrong: ', leak_cols)
#             if dim >= 3:
#                 return False
#         else:
#             has_equal_row_pair = True
    return True

def find_leak_col_use_row_check(leak_cols, candidate_cols, leak_rows, all_leak_cols_set, head_tail_flag):
#     if len(leak_cols) == 1:
#         ## If need cold start, we should find max equal cnt columns
#         leak_cols = find_leak_col(leak_cols, candidate_cols, leak_rows)
#     if len(leak_cols) == 1:
#         return leak_cols
    valid_df = train.loc[leak_rows]
    head_match = False
    tail_match = False
    for col in candidate_cols:
        leak_cols_set = set(leak_cols)
        if col in leak_cols_set or col in all_leak_cols_set:
            continue
        
        if head_tail_flag == 0 or head_tail_flag == 2:
            new_leak_cols = [col] + leak_cols
            if check_leak_cols(valid_df, new_leak_cols[:3], leak_rows):
                head_match = True
                leak_cols = new_leak_cols
                print ('found head: ', col)
        
        if head_tail_flag == 1 or head_tail_flag == 2:
            new_leak_cols = leak_cols + [col]
            if check_leak_cols(valid_df, new_leak_cols[-3:], leak_rows):
                tail_match = True
                leak_cols = new_leak_cols
                print ('found tail: ', col)
    return leak_cols, head_match, tail_match

In [4]:
# for leak_cols in LEAK_LIST:
#     if not (check_leak_cols(train, leak_cols, LEAK_ROWS)):
#         print (leak_cols)
all_leak_cols = [col for cols in LEAK_LIST for col in cols]
len(set(all_leak_cols))

2600

In [10]:
local_leak_list = []
i = 0
all_leak_cols = ["target"] + [col for cols in LEAK_LIST for col in cols]
all_leak_cols_set = set(all_leak_cols)

candidate_cols = []
for col in train.columns:
    if col not in all_leak_cols_set:
        candidate_cols.append(col)
print ('candidate_cols number: ', len(candidate_cols))
# head_dict = {}
# tail_dict = {}
uniq_vals = train.loc[LEAK_ROWS[0], candidate_cols].unique()
eq_cnt_dict = {}
for val in uniq_vals:
    if val == 0:
        cnt = train.loc[LEAK_ROWS, candidate_cols].apply(lambda x: len(x[x == val]))
        eq_cnt_dict[val] = cnt[cnt != 0].shape
val_array = pd.Series(eq_cnt_dict).sort_values(ascending = False).index.values

candidate_cols number:  2391


In [12]:
for val in val_array:
    if val == 0:
        eq_cnt = train.loc[LEAK_ROWS, candidate_cols].apply(lambda x: len(x[x == val]))
        eq_cnt_nz_len = len(eq_cnt[eq_cnt != 0])
        if True: # :
            print (val, eq_cnt.max(), eq_cnt_nz_len)
            leak_cols_candidate = eq_cnt[eq_cnt != 0].sort_values(ascending = True).index.values
            for col in leak_cols_candidate:
                if col in all_leak_cols_set:
                    continue
                print ('start col: ', col)
                local_leak_cols = [col]
                previous_leak_len = len(local_leak_cols)
                # search from columns from fill cnt
                while True:
                    local_leak_cols, _, _ = find_leak_col_use_row_check(local_leak_cols, leak_cols_candidate, 
                                                                  LEAK_ROWS, all_leak_cols_set, 2)
                    if len(local_leak_cols) == previous_leak_len:
                        print ('search ', len(local_leak_cols), ' cols from columns from fill cnt')
                        break
                    previous_leak_len = len(local_leak_cols)
                # pure search
                previous_leak_len = len(local_leak_cols)
                
                head_tail_flag = 2
                while True:
                    local_leak_cols, head_match, tail_match = find_leak_col_use_row_check(local_leak_cols, candidate_cols, 
                                                                  LEAK_ROWS, all_leak_cols_set, head_tail_flag)
                    if head_match and tail_match:
                        head_tail_flag = 2
                    elif head_match:
                        head_tail_flag = 0
                    elif tail_match:
                        head_tail_flag = 1
                    else:
                        break
                local_leak_list.append((local_leak_cols, val))
                all_leak_cols += local_leak_cols
                all_leak_cols_set = set(all_leak_cols)
                print ('Leak cols: ', local_leak_cols)
    if i % 10 == 0:
        print ('process vals: ', i)
    i += 1
import pickle
with open(path + 'leak_cols_val_0.pickle', 'wb+') as handle:
    pickle.dump(local_leak_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
# train.loc[valid_index, leak_cols]
# valid_df[leak_col]
# train.loc[valid_index, eq_cnt[eq_cnt != 0].sort_values().index]
# eq_cnt[eq_cnt != 0].sort_values()
# eq_cnt[eq_cnt != 0].sort_values()
# find_leak_col(eq_cnt[eq_cnt != 0].sort_values())
# train.loc[valid_rows.iloc[0].name, other_cols][train.loc[valid_rows.iloc[0].name, other_cols] != 0].unique

0.0 83 2391
start col:  50a900e26
found tail:  ca4eab5c5
found head:  4d9538272
found tail:  8c564ae48
found head:  f8de3e357
found head:  4bf2b8e7c
found tail:  16a9296fd
found head:  0892b3439
found tail:  9bed59a71
found head:  376474413
found head:  75d240f7b
found tail:  683d89bf1
found tail:  736fce873
found tail:  60b76e463
found tail:  6a3b5a968
found tail:  890163e1a
found tail:  2c136905e
found tail:  08d203407
found tail:  08af3dd45
found tail:  fbe52b1b2
found tail:  e2b4d4ef7
found tail:  78c239acf
found tail:  3da2882fd
found tail:  10596ddee
found tail:  2135da74a
found tail:  3e0e55648
found tail:  acee6ff41
found tail:  93c1eecb4
found tail:  b6e38a517
found tail:  e9c7ccc05
found tail:  2bf7dc91d
found tail:  06b19b6c4
found tail:  b0e45a9f7
found tail:  fd206ec4d
found tail:  c63090352
found tail:  df1ed6b50
found tail:  28dc3cc44
found tail:  7b1ddbabf
found tail:  5a88e3d89
found tail:  f2520b601
search  40  cols from columns from fill cnt
Leak cols:  ['75d240f7b',

found tail:  be90775f4
found tail:  651124842
found tail:  51d5e73a8
found tail:  8016f08af
found tail:  f80259ab3
found tail:  3685524f4
found tail:  532740e5d
found tail:  30347e683
found tail:  806dfdd51
search  40  cols from columns from fill cnt
Leak cols:  ['8677d6620', '75b846f12', '3a01b4018', '23d6be31e', '52695ed4a', 'ba9f3a42c', '135091a07', '19537e282', 'd5d4f936e', '578a07608', '63df94487', '169875559', 'b6ae5f5ca', '315b44e13', '5150b1a17', 'c8c6fe1a0', 'd918835ca', '8768af50f', '2cc11689d', '51c9aee7e', '188a6e279', '649d727e1', 'a8e878643', '8d4f4c571', 'f990bddac', '5719bbfc3', '12d3a67b0', '5f76b9c2f', 'c33a4095a', 'aac0c81ba', '2ba3b18ee', 'be90775f4', '651124842', '51d5e73a8', '8016f08af', 'f80259ab3', '3685524f4', '532740e5d', '30347e683', '806dfdd51']
start col:  e3fd6fa46
found tail:  53bba91b7
found head:  932b61d77
found head:  8ba7eacbb
found tail:  d24a55c98
found head:  7bde71e2f
found tail:  93f686d09
found head:  1b20c5c27
found head:  d40eb2705
found head

found tail:  98c05b5b4
found head:  422acfada
found tail:  8ff5cf8c3
found head:  0895b9597
found tail:  bf257a3bc
found head:  5d52b2d2b
found tail:  f86eabcb9
found tail:  c77f6dbfe
found head:  d9fbf30dd
found head:  81f4bc3d8
found tail:  014574363
found head:  184166328
found head:  2e22a32fd
found tail:  bf2b3327b
found tail:  74f15ad1c
found head:  357784061
found tail:  7d9979702
found head:  0989e152f
found head:  433d408b2
found head:  d974e8584
found head:  afc08501b
found head:  638d0b2e0
found head:  da59aa67e
found head:  2585a26fe
found head:  d7db08081
found head:  06283f348
found head:  73eeb092f
found head:  3b67e9641
found head:  398cdc105
found head:  d4087a281
found head:  232ecc840
found head:  f6240919f
found head:  f29217d2f
found head:  006e72749
found head:  0b2e88d2d
found head:  175891ff4
found head:  4fc9da4e3
found head:  81e3fed66
found head:  e1c250dbf
search  40  cols from columns from fill cnt
Leak cols:  ['e1c250dbf', '81e3fed66', '4fc9da4e3', '175891

found head:  f5c5884de
found tail:  78f45daba
found tail:  baccae68e
found head:  ce724eda8
found head:  ea3f3029c
found tail:  ea3f3029c
found head:  54df5e66f
found tail:  7c482420c
found head:  0f73ff7fa
found tail:  fa9e059a1
found tail:  9f2619218
found tail:  8e0dce2be
found tail:  70d96c477
found tail:  4563c7ee5
found tail:  1413972fe
found tail:  2ce7f870e
found tail:  b97f08b24
found tail:  504c7e3bb
found tail:  bfc7c09df
found tail:  9f57a28e7
found tail:  42481ff40
search  41  cols from columns from fill cnt
Leak cols:  ['0f73ff7fa', '54df5e66f', 'ea3f3029c', 'ce724eda8', 'f5c5884de', '018f23166', '6fc3be3e7', 'f227c1b60', 'a4151c1dc', 'fbc6c2b78', '2fbc7a055', 'b9e8c09d1', '67a590e8a', 'd30f53496', 'ebfc53bb0', 'ded346ef9', 'a2bd73c67', '10d7e8a17', 'b2a22ec15', 'b21f5b472', 'a05794fa5', 'a246962f5', '7a91cb0bc', '0554bac3e', 'f49c1d119', '78f45daba', 'baccae68e', 'ea3f3029c', '7c482420c', 'fa9e059a1', '9f2619218', '8e0dce2be', '70d96c477', '4563c7ee5', '1413972fe', '2ce7

KeyboardInterrupt: 

In [ ]:
# train.loc[leak_rows, ['2e7f340f2', '22f95560c']]
# leak_cols = LEAK_LIST[0]
# lag_list = []
# for i in range(len(LEAK_ROWS) - 1):
#     r = LEAK_ROWS[i]
#     r_next = LEAK_ROWS[i + 1]
#     if row_equal(train.loc[LEAK_ROWS, leak_cols], r, r_next, 1):
#     #             print ('Leak cols are wrong: ', leak_cols)
#         lag_list.append(1)
#     else:
#         lag_list.append(2)
uniq_vals = train.loc[LEAK_ROWS[0], candidate_cols].unique()
eq_cnt = {}
for val in uniq_vals:
    if val != 0:
        cnt = train.loc[LEAK_ROWS, candidate_cols].apply(lambda x: len(x[x == val]))
        eq_cnt[val] = cnt[cnt != 0].shape

In [45]:
pd.Series(eq_cnt).sort_values(ascending = False).index

Float64Index([  400000.0,   200000.0,  4000000.0,  1500000.0,  2000000.0,
                360000.0,   120000.0,    80000.0,  3000000.0,   790000.0,
                144000.0,  1032000.0,  1674000.0,  1540000.0,  2400000.0,
                174000.0,  5600000.0,   346000.0,  4600000.0,   278000.0,
                182000.0,  2202500.0,   210000.0,  812666.66,  4677000.0,
                604000.0,  1830000.0, 7255906.66, 34714000.0],
             dtype='float64')

In [237]:
# eq_cnt = train.loc[valid_index, other_cols].apply(lambda x: len(x[x == 1674000.0]))
# eq_cols = eq_cnt[eq_cnt != 0].sort_values().index.values #['29ab304b9', 'bd8f989f1', '74d7998d4', 'f58fb412c', 'c7775aabf', '8a088af55', '4824c1e90', '7a7da3079', '20604ed8f', 'c976a87ad', 'a1db86e3b', '50e4f96cf', '18c35d2ea', '0c8063d63', '4ead853dc', 'b599b0064', '58ed8fb53', 'fd99c18b5', 'a75d400b8', 'ff65215db', 'beb7f98fd', '47cd6e6e4', '1d9078f84', 'd83a2b684', '64e483341', '5dca793da', 'fb5a3097e', '8c0a1fa32', '9884166a7', '5f341a818', 'd26279f1a', 'db147ffca', '4fe8154c8', '400e9303d', '762cbd0ab', 'f514fdb2e']
# print (eq_cols)
def find_leak_row(leak_rows, leak_cols):
    leak_rows_set = set(leak_rows)
    leak_cols_df = train[leak_cols]
    for r in train.index:
        if r in leak_rows_set:
            continue
        
        if row_equal(leak_cols_df, leak_rows[-1], r, 1):
            print ('found tail', r)
            leak_rows += [r]
        elif row_equal(leak_cols_df, leak_rows[-1], r, 2):
            print ('found tail', r)
            leak_rows += [r]
            
        if row_equal(leak_cols_df, r, leak_rows[0], 1):
            print ('found head', r)
            leak_rows = [r] + leak_rows
        elif row_equal(leak_cols_df, r, leak_rows[0], 2):
            print ('found head', r)
            leak_rows = [r] + leak_rows
    return leak_rows

local_leak_cols = top_cols #['f0317ca4f', '402b0d650', '7e78d546b', '2ad744c57', '47abb3cb4', '71ac2b961', '5b8c88c94', '293e2698e', '4bdeca0d2', '2ef8b7f4f', 'c380056bb', '2488e17f5', '20442bac4', '8e8736fc8', '8a4c53d3e', '62c547c8e', '86f13324d', 'da52febdb', '64e38e7a2', 'b0310a768', '0d866c3d7', '34a2f580b', '24bcc2f15', 'e1e8947d8', '05f11f48f', '8c8616b62', '79e0c374a', 'ad1466df8', 'f642213a6', 'f8405f8b9', '1ae0db9d5', '9dbb6b717', '0f7ae26ce', '81ec47b4c', 'ad4e33a4c', 'a78f85d49', '8de6fcbf1', '3aab2691c', 'd2ef684ed', '9e39c29d0']
# while len(leak_cols) < 40:
#     leak_cols = find_leak_col(leak_cols, other_cols)
# print (leak_cols)
# local_leak_cols = find_leak_col(local_leak_cols, other_cols)
# local_leak_cols
# valid_index = valid_rows.index.copy()
# leak_rows = list(valid_index.values)
# valid_rows
leak_rows_len = len(leak_rows)
while True:
    leak_rows = find_leak_row(leak_rows, local_leak_cols)
    if len(leak_rows) == leak_rows_len:
        break
    leak_rows_len = len(leak_rows)
# leak_rows
train.loc[leak_rows, local_leak_cols]

found tail f51433ccc


,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,20aa07010,024c577b9,d6bb78916,b43a7cfd5,58232a6fb,1702b5bf0,324921c7b,62e59a501,2ec5b290f,241f0f867,fb49e4212,66ace2992,f74e8f13d,5c6487af1,963a49cdc,26fc93eb7,1931ccfdd,703885424,70feb1494,491b9ee45,23310aa6f,e176a204a,6619d81fc,1db387535,fc99f9426,91f701ba2,0572565c2,190db8488,adb64ff71,c47340d97,c5a231d81,0ff32eb98
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6726fff18,1015000.00,1.563412e+06,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
d94655f86,540000.00,1.015000e+06,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
1df3ca92e,115636.36,5.400000e+05,1.015000e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
d8e48b069,834800.00,1.156364e+05,5.400000e+05,1015000.00,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
24204cd10,296444.44,8.348000e+05,1.156364e+05,540000.00,1.015000e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
bf6c2d1ef,247166.66,2.964444e+05,8.348000e+05,115636.36,5.400000e+05,1015000.00,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
eacc7ab9e,550000.00,2.471667e+05,2.964444e+05,834800.00,1.156364e+05,540000.00,1.015000e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4a5425356,440000.00,3.076667e+06,5.500000e+05,247166.66,2.964444e+05,834800.00,1.156364e+05,540000.00,1.015000e+06,1563411.76,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.

In [239]:
previous_leak_len = len(local_leak_cols)
# search from columns from fill cnt
while True:
    local_leak_cols = find_leak_col(local_leak_cols, other_cols, leak_rows)
    if len(local_leak_cols) == previous_leak_len:
        break
    previous_leak_len = len(local_leak_cols)

found head target max_head_cnt 29


In [240]:
pd.set_option('max_rows', None)
train.loc[leak_rows, top_cols]

,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,20aa07010,024c577b9,d6bb78916,b43a7cfd5,58232a6fb,1702b5bf0,324921c7b,62e59a501,2ec5b290f,241f0f867,fb49e4212,66ace2992,f74e8f13d,5c6487af1,963a49cdc,26fc93eb7,1931ccfdd,703885424,70feb1494,491b9ee45,23310aa6f,e176a204a,6619d81fc,1db387535,fc99f9426,91f701ba2,0572565c2,190db8488,adb64ff71,c47340d97,c5a231d81,0ff32eb98
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6726fff18,1015000.00,1.563412e+06,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
d94655f86,540000.00,1.015000e+06,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
1df3ca92e,115636.36,5.400000e+05,1.015000e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
d8e48b069,834800.00,1.156364e+05,5.400000e+05,1015000.00,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
24204cd10,296444.44,8.348000e+05,1.156364e+05,540000.00,1.015000e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
bf6c2d1ef,247166.66,2.964444e+05,8.348000e+05,115636.36,5.400000e+05,1015000.00,1.563412e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
eacc7ab9e,550000.00,2.471667e+05,2.964444e+05,834800.00,1.156364e+05,540000.00,1.015000e+06,1563411.76,1.563412e+06,1563411.76,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4a5425356,440000.00,3.076667e+06,5.500000e+05,247166.66,2.964444e+05,834800.00,1.156364e+05,540000.00,1.015000e+06,1563411.76,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1.563412e+06,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.

In [252]:
from leak_rows import *
from leak_cols import *

def row_equal(df, r1, r2, lag):
    return np.all(df.loc[r1].iloc[:-(lag)].values == df.loc[r2].iloc[lag:].values)
        
def check_leak_cols(df, leak_cols, leak_rows):
    for i in range(len(leak_rows) - 1):
        r = leak_rows[i]
        r_next = leak_rows[i + 1]
        if not (row_equal(df[leak_cols], r, r_next, 1) or row_equal(df[leak_cols], r, r_next, 2)):
#             print ('Leak cols are wrong: ', leak_cols)
            return False
    return True

for i in range(len(leak_list)):
    if not check_leak_cols(train, leak_list[i], leak_rows):
        print (i)

32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
54
55
56
57
58
59
60
61
62
63
64
65
66
67
71
72


In [243]:
len(set(leak_rows))

83

In [196]:
from leak_rows import leak_list

leak_cols = []
tot_len = 0
i = 0
for cols in leak_list:
    tot_len += len(cols)
    print (i, len(cols))
    leak_cols += cols
    if i == 31:
        break
    i += 1
print(tot_len, len(set(leak_cols)))
# i = 31
# print(len(leak_list[i]))
# train.loc[valid_index, leak_list[i]]
# leak_col = leak_list[i]
# find_leak_col(leak_col, other_cols)
# val = 45138000
# train_leaks = train[leak_list[i]].apply(lambda x: len(x[x == val]), axis = 1)
# train_leaks[train_leaks != 0].sort_values(ascending = False).shape

0 40
1 40
2 40
3 40
4 40
5 40
6 40
7 40
8 40
9 40
10 40
11 40
12 40
13 40
14 40
15 40
16 40
17 40
18 40
19 40
20 40
21 40
22 40
23 40
24 40
25 40
26 40
27 40
28 40
29 41
30 40
31 40
1281 1281
